# Data unit tests
fazer expectations, criar datasource, data_assets e checkpoints

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import torch
import logging
from typing import Any, Optional
from ydata_profiling.expectations_report import ExpectationsReport
import pandas as pd
from visions import VisionsTypeset
import hopsworks
import pandas as pd
from datetime import datetime
from typing import Optional
import hsfs
from great_expectations.core import ExpectationSuite
from hsfs.feature_group import FeatureGroup
from ydata_profiling.config import Settings
from ydata_profiling.model import BaseDescription, expectation_algorithms
from ydata_profiling.model.handler import Handler
from ydata_profiling.utils.dataframe import slugify
from great_expectations.checkpoint import SimpleCheckpoint
import torch.nn as nn
from ydata_profiling import ProfileReport
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve
from sklearn.model_selection import cross_val_score
import great_expectations as gx
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('heart.csv')

In [4]:
df

age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0     63    1   3     145   233    1        0       150     0      2.3    0   
1     37    1   2     130   250    0        1       187     0      3.5    0   
2     41    0   1     130   204    0        0       172     0      1.4    2   
3     56    1   1     120   236    0        1       178     0      0.8    2   
4     57    0   0     120   354    0        1       163     1      0.6    2   
..   ...  ...  ..     ...   ...  ...      ...       ...   ...      ...  ...   
298   57    0   0     140   241    0        1       123     1      0.2    1   
299   45    1   3     110   264    0        1       132     0      1.2    1   
300   68    1   0     144   193    1        1       141     0      3.4    1   
301   57    1   0     130   131    0        1       115     1      1.2    1   
302   57    0   1     130   236    0        0       174     0      0.0    1   

     caa  thall  output  
0      0      1       1  
1      0      2       1  
2      0      2       1  
3      0      2       1  
4      0      2       1  
..   ...    ...     ...  
298    0      3       0  
299    0      3       0  
300    2      3       0  
301    1      3       0  
302    1      2       0  

[303 rows x 14 columns]

In [5]:
split_point = int(0.7 * len(df))

df_ref = df[:split_point]  # Reference dataset (first 70%)
df_ana = df[split_point:]  # Analysis dataset (remaining 30%)

In [6]:
#!tree gx

In [7]:
context = gx.get_context(context_root_dir = "gx")

2024-06-21 00:13:44,385 INFO: FileDataContext loading fluent config
2024-06-21 00:13:44,399 INFO: Loading 'datasources' ->
[{'assets': [...], 'name': 'hospital_datasource', 'type': 'pandas'}]
2024-06-21 00:13:44,403 INFO: _PandasDataAsset.dict() - missing `config_provider`, skipping config substitution
2024-06-21 00:13:44,497 INFO: Loaded 'hospital_datasource' from fluent config
2024-06-21 00:13:44,498 INFO: Saving 1 Fluent Datasources to C:\Users\pcata\MLOPS project\gx\great_expectations.yml
2024-06-21 00:13:44,499 INFO: DataFrameAsset.dict() - missing `config_provider`, skipping config substitution
2024-06-21 00:13:44,499 INFO: PandasDatasource.dict() - missing `config_provider`, skipping config substitution


In [ ]:
datasource_name = "hospital_datasource"
try:
    datasource = context.sources.add_pandas(datasource_name)
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

In [ ]:
print("data source", context.list_datasources())

In [ ]:
#suit bank é basicamente um conjunto de expextations que vamos criar nos dados de modo a que depois possam verificar novos dados.

In [ ]:
suite_bank = context.add_or_update_expectation_suite(expectation_suite_name="Hospital")

In [ ]:
df_ref.columns

In [ ]:
df_ref.dtypes

In [ ]:
for column in df_ref.columns:
    unique_values = df_ref[column].unique()
    print(f"Column '{column}' contains the following unique values: {unique_values}")

In [ ]:
expectation_age = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_between",
    kwargs={
        "column": "age",
        "max_value": 115,
        "min_value": 0
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_age)

In [ ]:
expectation_sex = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "sex",
        "value_set" : [0,1]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_sex)

In [ ]:
expectation_cp = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "cp",
        "value_set" : [3, 2, 1, 0]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_cp)

In [ ]:
expectation_trtbps = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_between",
    kwargs={
        "column": "trtbps",
        "max_value": 196,
        "min_value": 80
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_trtbps)

In [ ]:
expectation_chol = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_between",
    kwargs={
        "column": "chol",
        "max_value": 600,
        "min_value": 30
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_chol)

## Um homem de 50 anos com hipercolesterolemia familiar: Apresentou colesterol total de 1.100 mg/dL e LDL (colesterol "mau") de 800 mg/dL.
## Uma mulher de 30 anos com doença hepática: Registrou colesterol total de 1.200 mg/dL e LDL de 950 mg/dL.
## Um jovem de 20 anos com hipotireoidismo não tratado: Teve colesterol total de 1.400 mg/dL e LDL de 1.100 mg/dL.

In [ ]:
expectation_fbs = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "fbs",
        "value_set" : [0,1]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_fbs)

In [ ]:
expectation_restecg = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "restecg",
        "value_set" : [0,1,2]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_restecg)

In [ ]:
expectation_thalachh = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_between",
    kwargs={
        "column": "thalachh",
        "max_value": 290,
        "min_value": 50
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_thalachh)

In [ ]:
expectation_exng = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "exng",
        "value_set" : [0,1]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_exng)

In [ ]:
expectation_oldpeak = ExpectationConfiguration(
    expectation_type="expect_column_values_to_be_between",
    kwargs={
        "column": "oldpeak",
        "max_value": 7.0,
        "min_value": 0.0
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_oldpeak)

In [ ]:
expectation_slp = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "slp",
        "value_set" : [0,1,2]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_slp)

In [ ]:
expectation_caa = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "caa",
        "value_set" : [0,1,2,3,4]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_caa)

In [ ]:
expectation_thall = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "thall",
        "value_set" : [0,1,2,3]
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_thall)

In [ ]:
'''expectation_output = ExpectationConfiguration(
    expectation_type="expect_column_distinct_values_to_be_in_set",
    kwargs={
        "column": "output",
        "value_set" : ['0','1']
    },
)
suite_bank.add_expectation(expectation_configuration=expectation_marital)'''
#é suposto fazer isso para a var output? acho que nao

In [ ]:
context.save_expectation_suite(expectation_suite=suite_bank)

In [ ]:
data_asset_name = "Reference Data"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe=df_ref)
except:
    print("The data asset already exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

In [ ]:
batch_request = data_asset.build_batch_request(dataframe=df_ref)

In [ ]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_expectations",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "Hospital",
        },
    ],
)
checkpoint_result = checkpoint.run()

In [ ]:
def get_validation_results(checkpoint_result):
    # validation_result is a dictionary containing one key-value pair
    validation_result_key, validation_result_data = next(iter(checkpoint_result["run_results"].items()))

    # Accessing the 'actions_results' from the validation_result_data
    validation_result_ = validation_result_data.get('validation_result', {})

    # Accessing the 'results' from the validation_result_data
    results = validation_result_["results"]
    meta = validation_result_["meta"]
    use_case = meta.get('expectation_suite_name')
    
    
    df_validation = pd.DataFrame({},columns=["Success","Expectation Type","Column","Column Pair","Max Value",\
                                       "Min Value","Element Count","Unexpected Count","Unexpected Percent","Value Set","Unexpected Value","Observed Value"])
    
    
    for result in results:
        # Process each result dictionary as needed
        success = result.get('success', '')
        expectation_type = result.get('expectation_config', {}).get('expectation_type', '')
        column = result.get('expectation_config', {}).get('kwargs', {}).get('column', '')
        column_A = result.get('expectation_config', {}).get('kwargs', {}).get('column_A', '')
        column_B = result.get('expectation_config', {}).get('kwargs', {}).get('column_B', '')
        value_set = result.get('expectation_config', {}).get('kwargs', {}).get('value_set', '')
        max_value = result.get('expectation_config', {}).get('kwargs', {}).get('max_value', '')
        min_value = result.get('expectation_config', {}).get('kwargs', {}).get('min_value', '')

        element_count = result.get('result', {}).get('element_count', '')
        unexpected_count = result.get('result', {}).get('unexpected_count', '')
        unexpected_percent = result.get('result', {}).get('unexpected_percent', '')
        observed_value = result.get('result', {}).get('observed_value', '')
        if type(observed_value) is list:
            #sometimes observed_vaue is not iterable
            unexpected_value = [item for item in observed_value if item not in value_set]
        else:
            unexpected_value=[]
        
        df_validation = pd.concat([df_validation, pd.DataFrame.from_dict( [{"Success" :success,"Expectation Type" :expectation_type,"Column" : column,"Column Pair" : (column_A,column_B),"Max Value" :max_value,\
                                           "Min Value" :min_value,"Element Count" :element_count,"Unexpected Count" :unexpected_count,"Unexpected Percent":unexpected_percent,\
                                                  "Value Set" : value_set,"Unexpected Value" :unexpected_value ,"Observed Value" :observed_value}])], ignore_index=True)
        
    return df_validation

In [ ]:
df_validation = get_validation_results(checkpoint_result)

In [ ]:
df_validation

In [ ]:
profile = ProfileReport(df_ref, title=f"Hospital Profiling Report", minimal=True)

In [ ]:
profile

In [ ]:
class ExpectationHandler(Handler):
    """Default handler"""

    def __init__(self, typeset: VisionsTypeset, *args, **kwargs):
        mapping = {
            "Unsupported": [expectation_algorithms.generic_expectations],
            "Text": [expectation_algorithms.categorical_expectations],
            "Categorical": [expectation_algorithms.categorical_expectations],
            "Boolean": [expectation_algorithms.categorical_expectations],
            "Numeric": [expectation_algorithms.numeric_expectations],
            "URL": [expectation_algorithms.url_expectations],
            "File": [expectation_algorithms.file_expectations],
            "Path": [expectation_algorithms.path_expectations],
            "DateTime": [expectation_algorithms.datetime_expectations],
            "Image": [expectation_algorithms.image_expectations],
        }
        super().__init__(mapping, typeset, *args, **kwargs)


class ExpectationsReportV3:
    config: Settings
    df: Optional[pd.DataFrame] = None

    @property
    def typeset(self) -> Optional[VisionsTypeset]:
        return None

    def to_expectation_suite(
        self,
        datasource_name,
        data_asset_name,
        suite_name: Optional[str] = None,
        data_context: Optional[Any] = None,
        save_suite: bool = True,
        run_validation: bool = True,
        build_data_docs: bool = True,
        handler: Optional[Handler] = None,
    ) -> Any:
        """
        All parameters default to True to make it easier to access the full functionality of Great Expectations out of
        the box.
        Args:
            suite_name: The name of your expectation suite
            data_context: A user-specified data context
            save_suite: Boolean to determine whether to save the suite to .json as part of the method
            run_validation: Boolean to determine whether to run validation as part of the method
            build_data_docs: Boolean to determine whether to build data docs, save the .html file, and open data docs in
                your browser
            handler: The handler to use for building expectation

        Returns:
            An ExpectationSuite
        """
        try:
            import great_expectations as ge
        except ImportError as ex:
            raise ImportError(
                "Please install great expectations before using the expectation functionality"
            ) from ex

        # Use report title if suite is empty
        if suite_name is None:
            suite_name = slugify(self.config.title)

        # Use the default handler if none
        if handler is None:
            handler = ExpectationHandler(self.typeset)

        # Obtain the ge context and create the expectation suite
        if not data_context:
            data_context = ge.data_context.DataContext()

        data_asset = data_context.get_datasource(datasource_name).get_asset(data_asset_name)

        batch_request = data_asset.build_batch_request()

        suite = data_context.add_or_update_expectation_suite(expectation_suite_name=suite_name)


        # Instantiate an in-memory pandas dataset
        validator = data_context.get_validator(batch_request=batch_request, expectation_suite=suite)

        # Obtain the profiling summary
        summary: BaseDescription = self.get_description()  # type: ignore

        # Dispatch to expectations per semantic variable type
        for name, variable_summary in summary.variables.items():
            handler.handle(variable_summary["type"], name, variable_summary, validator)

        # We don't actually update the suite object on the batch in place, so need
        # to get the populated suite from the batch
        suite = validator.get_expectation_suite(discard_failed_expectations=False)
        data_context.update_expectation_suite(suite)
        
        validation_result_identifier = None
        if run_validation:
            checkpoint_config = {
                "class_name": "SimpleCheckpoint",
                "validations": [
                    {
                        "batch_request": batch_request,
                        "expectation_suite_name": suite_name,
                    }
                ]
            }
            checkpoint = SimpleCheckpoint(
                f"_tmp_checkpoint_{suite_name}",
                data_context,
                suite,
                **checkpoint_config,

            )
            results = checkpoint.run(result_format="SUMMARY", run_name=suite_name)
            validation_result_identifier = results.list_validation_result_identifiers()[0]



        # Write expectations and open data docs
        if save_suite or build_data_docs:
            data_context.update_expectation_suite(suite)

        if build_data_docs:
            data_context.build_data_docs()


        return validator.get_expectation_suite()

In [ ]:
ExpectationsReport.to_expectation_suite = ExpectationsReportV3.to_expectation_suite

In [ ]:
datasource_name = "hospital_datasource"

In [ ]:
new_suite_hospital = profile.to_expectation_suite(
    datasource_name = datasource_name,
    data_asset_name="Reference Data",
    suite_name = 'New Reference Data',
    data_context=context
)

In [ ]:
context.save_expectation_suite(expectation_suite=new_suite_hospital)

In [ ]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_full",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "New Reference Data",
        },
    ],
)
checkpoint_result = checkpoint.run()

In [ ]:
df_validation = get_validation_results(checkpoint_result)

In [ ]:
df_validation

### Este codigo aqui em baixo é a mesma coisa que fiz desde o inicio mas em apenas uma celula.
###Primeiro codigo a correr

In [4]:
import pandas as pd
import great_expectations as gx
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.checkpoint import SimpleCheckpoint
import logging
from ydata_profiling import ProfileReport

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Read the data
df = pd.read_csv('heart.csv')

# Split the data
split_point = int(0.7 * len(df))
df_ref = df[:split_point]  # Reference dataset (first 70%)
df_ana = df[split_point:]  # Analysis dataset (remaining 30%)

# Initialize Great Expectations context
context = gx.get_context(context_root_dir="gx")
datasource_name = "hospital_datasource"

try:
    datasource = context.sources.add_pandas(datasource_name)
except Exception as e:
    logger.warning(f"Data Source already exists or error: {e}")
    datasource = context.datasources[datasource_name]

logger.info(f"Data source: {context.list_datasources()}")

# Create expectation suite
suite_name = "Hospital"

def create_expectation_suite(context, suite_name):
    try:
        suite = context.create_expectation_suite(expectation_suite_name=suite_name)
    except Exception as e:
        logger.error(f"Failed to create expectation suite: {e}")
        return None

    expectations = [
        ExpectationConfiguration("expect_column_values_to_be_between", {"column": "age", "min_value": 0, "max_value": 115}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "sex", "value_set": [0, 1]}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "cp", "value_set": [0, 1, 2, 3]}),
        ExpectationConfiguration("expect_column_values_to_be_between", {"column": "trtbps", "min_value": 80, "max_value": 196}),
        ExpectationConfiguration("expect_column_values_to_be_between", {"column": "chol", "min_value": 30, "max_value": 600}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "fbs", "value_set": [0, 1]}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "restecg", "value_set": [0, 1, 2]}),
        ExpectationConfiguration("expect_column_values_to_be_between", {"column": "thalachh", "min_value": 50, "max_value": 290}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "exng", "value_set": [0, 1]}),
        ExpectationConfiguration("expect_column_values_to_be_between", {"column": "oldpeak", "min_value": 0.0, "max_value": 7.0}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "slp", "value_set": [0, 1, 2]}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "caa", "value_set": [0, 1, 2, 3, 4]}),
        ExpectationConfiguration("expect_column_distinct_values_to_be_in_set", {"column": "thall", "value_set": [0, 1, 2, 3]})
    ]

    for exp in expectations:
        suite.add_expectation(expectation_configuration=exp)

    context.save_expectation_suite(suite)
    return suite

suite_bank = create_expectation_suite(context, suite_name)

# Validate data against the expectation suite
def validate_data(context, suite_name, df):
    try:
        datasource = context.datasources["hospital_datasource"]
        data_asset = datasource.get_asset("Reference Data")
    except KeyError:
        logger.error("Datasource or data asset not found.")
        return None

    batch_request = data_asset.build_batch_request(dataframe=df)

    checkpoint = SimpleCheckpoint(
        name="checkpoint_expectations",
        data_context=context,
        validations=[{"batch_request": batch_request, "expectation_suite_name": suite_name}]
    )

    result = checkpoint.run()
    return result

def get_validation_results(checkpoint_result):
    validation_result_data = list(checkpoint_result["run_results"].values())[0]["validation_result"]
    results = validation_result_data["results"]
    validation_data = []

    for result in results:
        row = {
            "Success": result["success"],
            "Expectation Type": result["expectation_config"]["expectation_type"],
            "Column": result["expectation_config"]["kwargs"].get("column"),
            "Max Value": result["expectation_config"]["kwargs"].get("max_value"),
            "Min Value": result["expectation_config"]["kwargs"].get("min_value"),
            "Value Set": result["expectation_config"]["kwargs"].get("value_set"),
            "Unexpected Value": result["result"].get("unexpected_value"),
            "Observed Value": result["result"].get("observed_value")
        }
        validation_data.append(row)
    
    df_validation = pd.DataFrame(validation_data)
    return df_validation

# Validate the data
checkpoint_result = validate_data(context, suite_name, df_ref)
df_validation = get_validation_results(checkpoint_result)
df_validation

# Generate profiling report
profile = ProfileReport(df_ref, title="Hospital Profiling Report", minimal=True)
profile.to_notebook_iframe()

2024-06-07 23:59:44,787 INFO: FileDataContext loading fluent config
2024-06-07 23:59:44,801 INFO: Loading 'datasources' ->
[{'assets': [...], 'name': 'hospital_datasource', 'type': 'pandas'}]
2024-06-07 23:59:44,802 INFO: _PandasDataAsset.dict() - missing `config_provider`, skipping config substitution
2024-06-07 23:59:44,878 INFO: Loaded 'hospital_datasource' from fluent config
2024-06-07 23:59:44,880 INFO: Saving 1 Fluent Datasources to C:\Users\pcata\MLOPS project\gx\great_expectations.yml
2024-06-07 23:59:44,881 INFO: DataFrameAsset.dict() - missing `config_provider`, skipping config substitution
2024-06-07 23:59:44,882 INFO: PandasDatasource.dict() - missing `config_provider`, skipping config substitution
2024-06-07 23:59:44,913 WARNING: Data Source already exists or error: Can not write the fluent datasource hospital_datasource because a datasource of that name already exists in the data context.
2024-06-07 23:59:44,914 INFO: DataFrameAsset.dict() - missing `config_provider`, ski

Calculating Metrics:   0%|          | 0/46 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
df_validation

Success                            Expectation Type    Column  Max Value  \
0      True          expect_column_values_to_be_between       age      115.0   
1      True  expect_column_distinct_values_to_be_in_set       sex        NaN   
2      True  expect_column_distinct_values_to_be_in_set        cp        NaN   
3      True          expect_column_values_to_be_between    trtbps      196.0   
4      True          expect_column_values_to_be_between      chol      600.0   
5      True  expect_column_distinct_values_to_be_in_set       fbs        NaN   
6      True  expect_column_distinct_values_to_be_in_set   restecg        NaN   
7      True          expect_column_values_to_be_between  thalachh      290.0   
8      True  expect_column_distinct_values_to_be_in_set      exng        NaN   
9      True          expect_column_values_to_be_between   oldpeak        7.0   
10     True  expect_column_distinct_values_to_be_in_set       slp        NaN   
11     True  expect_column_distinct_values_to_be_in_set       caa        NaN   
12     True  expect_column_distinct_values_to_be_in_set     thall        NaN   

    Min Value        Value Set Unexpected Value   Observed Value  
0         0.0             None             None             None  
1         NaN           [0, 1]             None           [0, 1]  
2         NaN     [3, 2, 1, 0]             None     [0, 1, 2, 3]  
3        80.0             None             None             None  
4        30.0             None             None             None  
5         NaN           [0, 1]             None           [0, 1]  
6         NaN        [0, 1, 2]             None        [0, 1, 2]  
7        50.0             None             None             None  
8         NaN           [0, 1]             None           [0, 1]  
9         0.0             None             None             None  
10        NaN        [0, 1, 2]             None        [0, 1, 2]  
11        NaN  [0, 1, 2, 3, 4]             None  [0, 1, 2, 3, 4]  
12        NaN     [0, 1, 2, 3]             None     [0, 1, 2, 3]

# Feature Store

In [ ]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): Name of the expectation suite.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    
    expectation_suite = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    
    # Client Features Expectations
    if feature_group == 'client_features':
        # Ensure age is an integer and within a plausible range
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "age", "type_": "int64"},
            )
        )
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_between",
                kwargs={"column": "age", "min_value": 0, "max_value": 120},
            )
        )

        # Ensure sex is either 0 or 1 (assuming binary representation)
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_in_set",
                kwargs={"column": "sex", "value_set": [0, 1]},
            )
        )

        # Ensure other integer columns are of type int64
        for column in ['cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'slp', 'caa', 'thall']:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": column, "type_": "int64"},
                )
            )
        
        # Ensure oldpeak is a float
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "oldpeak", "type_": "float64"},
            )
        )

        # Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_columns_to_match_ordered_list",
                kwargs={
                    "column_list": ["index", "age", "sex", "cp", "trtbps", "chol", "fbs", "restecg", "thalachh", "exng", "oldpeak", "slp", "caa", "thall", "output"]
                },
            )
        )
        
        # Nr of Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_column_count_to_equal", kwargs={"value": 15}
            )
        )

    # Target Features Expectations
    if feature_group == 'target_features':
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "output", "type_": "int64"},
            )
        ) 

        # Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_columns_to_match_ordered_list",
                kwargs={
                    "column_list": ["index", "output"]
                },
            )
        )        
        
        # Nr of Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_column_count_to_equal", kwargs={"value": 2}
            )
        )  

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_min_to_be_between",
                kwargs={
                    "column": "output",
                    "min_value": 0,
                    "strict_min": False,
                },
            )
        )
        
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "output"},
            )
        )
    
    # Health Features Expectations
    if feature_group == 'health_features':
        # Ensure all health-related columns are of the correct type
        for column in ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'slp', 'caa', 'thall']:
            expectation_suite.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_of_type",
                    kwargs={"column": column, "type_": "int64"},
                )
            )

        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "oldpeak", "type_": "float64"},
            )
        )

        # Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_columns_to_match_ordered_list",
                kwargs={
                    "column_list": ["index", "age", "sex", "cp", "trtbps", "chol", "fbs", "restecg", "thalachh", "exng", "oldpeak", "slp", "caa", "thall"]
                },
            )
        )
        
        # Nr of Columns
        expectation_suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_table_column_count_to_equal", kwargs={"value": 14}
            )
        )

    return expectation_suite

In [ ]:
# Building validation suites for each type of features
validation_expectation_suite_client = build_expectation_suite("client_expectations", "client_features")
validation_expectation_suite_target = build_expectation_suite("target_expectations", "target_features")
validation_expectation_suite_health = build_expectation_suite("health_expectations", "health_features")

In [ ]:
#!pip install python-dotenv

In [ ]:
import os
from pathlib import Path
from typing import Union
import datetime
from dotenv import load_dotenv

In [ ]:
def load_env_vars(root_dir: Union[str, Path]) -> dict:
    """
    Load environment variables from .env.default and .env files.\\

    Args:
        root_dir: Root directory of the .env files.

    Returns:
        Dictionary with the environment variables.
    """

    if isinstance(root_dir, str):
        root_dir = Path(root_dir)

    load_dotenv(dotenv_path=root_dir / ".env.default")
    load_dotenv(dotenv_path=root_dir / ".env", override=True)

    return dict(os.environ)

In [ ]:
def get_root_dir(default_value: str = ".") -> Path:
    """
    Get the root directory of the project.

    Args:
        default_value: Default value to use if the environment variable is not set.

    Returns:
        Path to the root directory of the project.
    """

    return Path(os.getenv("ML_PIPELINE_ROOT_DIR", default_value))

In [ ]:
def to_feature_store(
    data: pd.DataFrame,
    group_name: str,
    feature_group_version: int,
    description: str,
    group_description: dict,
    validation_expectation_suite: ExpectationSuite,
    SETTINGS: dict
):
    """
    This function takes in a pandas DataFrame and a validation expectation suite,
    performs validation on the data using the suite, and then saves the data to a
    feature store in the feature store.

    Args:
        data (pd.DataFrame): Dataframe with the data to be stored
        group_name (str): Name of the feature group.
        feature_group_version (int): Version of the feature group.
        description (str): Description for the feature group.
        group_description (dict): Description of each feature of the feature group. 
        validation_expectation_suite (ExpectationSuite): group of expectations to check data.
        SETTINGS (dict): Dictionary with the settings definitions to connect to the project.
        
    Returns:
        A dictionary with the feature view version, feature view name and training dataset feature version.
    
    
    """
    # Connect to feature store.
    project = hopsworks.login(
        api_key_value=SETTINGS["1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24"], project=SETTINGS["MLOps_Hospital"]
    )
    feature_store = project.get_feature_store()

    # Create feature group.
    object_feature_group = feature_store.get_or_create_feature_group(
        name=group_name,
        version=feature_group_version,
        description= description,
        primary_key=["index"],
        #event_time="datetime",
        online_enabled=False,
        expectation_suite=validation_expectation_suite,
    )
    # Upload data.
    object_feature_group.insert(
        features=data,
        overwrite=False,
        write_options={
            "wait_for_job": True,
        },
    )

    # Add feature descriptions.

    for description in group_description:
        object_feature_group.update_feature_description(
            description["name"], description["description"]
        )

    # Update statistics.
    object_feature_group.statistics_config = {
        "enabled": True,
        "histograms": True,
        "correlations": True,
    }
    object_feature_group.update_statistics_config()
    object_feature_group.compute_statistics()

    return object_feature_group


def to_feature_store(
    data: pd.DataFrame,
    group_name: str,
    feature_group_version: int,
    description: str,
    group_description: dict,
    validation_expectation_suite: ExpectationSuite,
    SETTINGS: dict
):
    """
    This function takes in a pandas DataFrame and a validation expectation suite,
    performs validation on the data using the suite, and then saves the data to a
    feature store in the feature store.

    Args:
        data (pd.DataFrame): Dataframe with the data to be stored
        group_name (str): Name of the feature group.
        feature_group_version (int): Version of the feature group.
        description (str): Description for the feature group.
        group_description (dict): Description of each feature of the feature group. 
        validation_expectation_suite (ExpectationSuite): group of expectations to check data.
        SETTINGS (dict): Dictionary with the settings definitions to connect to the project.
        
    Returns:
        A dictionary with the feature view version, feature view name and training dataset feature version.
    
    
    """
    # Connect to feature store.
    project = hopsworks.login(
        api_key_value=SETTINGS["1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24"], project=SETTINGS["MLOps_Hospital"]
    )
    feature_store = project.get_feature_store()

    # Create feature group.
    object_feature_group = feature_store.get_or_create_feature_group(
        name=group_name,
        version=feature_group_version,
        description= description,
        primary_key=["index"],
        #event_time="datetime",
        online_enabled=False,
        expectation_suite=validation_expectation_suite,
    )
    # Upload data.
    object_feature_group.insert(
        features=data,
        overwrite=False,
        write_options={
            "wait_for_job": True,
        },
    )

    # Add feature descriptions.

    for description in group_description:
        object_feature_group.update_feature_description(
            description["name"], description["description"]
        )

    # Update statistics.
    object_feature_group.statistics_config = {
        "enabled": True,
        "histograms": True,
        "correlations": True,
    }
    object_feature_group.update_statistics_config()
    object_feature_group.compute_statistics()

    return object_feature_group

In [ ]:
df_ref

In [ ]:
'''import pandas as pd
import hsfs

# Establish connection to the Hopsworks Feature Store
connection = hsfs.connection(
    host="https://c.app.hopsworks.ai/p/766100/view",
    project="MLOps_Hospital",
    engine="python",
    api_key_value="1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24"
)

# Get the feature store reference
fs = connection.get_feature_store(name='mlops_hospital_featurestore')

# Client Feature Group
client_fg = fs.create_feature_group(name="client_feature_group", version=1, primary_key=["index"])
client_fg.save(df_ref[['index', 'sex', 'age']])

# Health Feature Group
health_fg = fs.create_feature_group(name="health_feature_group", version=1, primary_key=["index"])
health_fg.save(df_ref.drop(columns=['sex', 'age', 'output']))

# Target Feature Group
target_fg = fs.create_feature_group(name="target_feature_group", version=1, primary_key=["index"])
target_fg.save(df_ref[['index', 'output']])'''

In [ ]:
connection = hsfs.connection(
    host="c.app.hopsworks.ai/p/766100/view",
    project="MLOps_Hospital",
    engine="python",
    api_key_value="1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24")
fs = connection.get_feature_store()
# 1. Explore Feature Groups
feature_groups = fs.get_feature_groups(name='client_feature_group')

# Print information about each feature group
for feature_group in feature_groups:
    print("Feature Group Name:", feature_group.name)
    print("Feature Group Description:", feature_group.description)
    print("Feature Group Version:", feature_group.version)
    print("Features in Feature Group:", feature_group.features)
    print("--------------------------------------------")

# 2. Query Features
feature_group = fs.get_feature_group("client_feature_group")  # Assuming "client_features" is the name of the feature group
features = feature_group.read()

# Print the first few rows of the features
print("First few rows of features:")
print(features.head())

In [ ]:
feature_groups_to_explore = ["health_feature_group", "target_feature_group"]

# 1. Explore Feature Groups
for group_name in feature_groups_to_explore:
    try:
        feature_groups = fs.get_feature_groups(name=group_name)

        for feature_group in feature_groups:
            print("Feature Group Name:", feature_group.name)
            print("Feature Group Description:", feature_group.description)
            print("Feature Group Version:", feature_group.version)
            print("Features in Feature Group:", [f.name for f in feature_group.features])
            print("--------------------------------------------")
    except Exception as e:
        print(f"An error occurred while accessing feature group {group_name}: {e}")

# 2. Query Features
for group_name in feature_groups_to_explore:
    try:
        feature_group = fs.get_feature_group(group_name)  # Assuming the latest version
        features = feature_group.read()

        # Print the first few rows of the features
        print(f"First few rows of features from {group_name}:")
        print(features.head())
    except Exception as e:
        print(f"An error occurred while querying features from feature group {group_name}: {e}")

In [ ]:
'''import pandas as pd
import hsfs

# Assuming df_ref is your DataFrame

# Add an index column to df_ref
df_ref['index'] = range(len(df_ref))

# Establish connection to the Hopsworks Feature Store
connection = hsfs.connection(
    host="https://c.app.hopsworks.ai/p/766100/view",
    port=443,
    project="MLOps_Hospital",
    engine="python",
    api_key_value="dBSTKOYRzA6O1Bk8c2pzM1Jq0j8bdIGd9RbbO3l7"
)

# Get the feature store reference
fs = connection.get_feature_store(name='mlops_hospital_featurestore')

# Create a feature group with primary key
fg = fs.create_feature_group(name="example_feature_group", version=1, primary_key=["index"])

# Save data from df_ref to the feature group
fg.save(df_ref)

# Read data from the feature group
features = fg.read()

# Print the first few rows of the features
print("First few rows of features:")
print(features.show(5))
'''

In [ ]:
'''# Get the existing feature group
fg = fs.get_feature_group(name="example_feature_group", version=1)

# Read data from the feature group
features = fg.read()

# Print the first few rows of the features
print("First few rows of features:")
features
'''

In [ ]:
cheque_fg = fs.get_feature_group("client_feature_group", version=1)
feature_view = fs.get_or_create_feature_view(
    name="client_feature_view",
    version=1,
    description='visualizar a data dos clients',
    query=cheque_fg.select(["sex","age"]),
)

In [ ]:
cheque_fg = fs.get_feature_group("health_feature_group", version=1)
feature_view = fs.get_or_create_feature_view(
    name="health_feature_view",
    version=1,
    description='visualizar a data dos clients sobre a saude',
    query=cheque_fg.select(['cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall', 'index']),
)

In [ ]:
cheque_fg = fs.get_feature_group("target_feature_group", version=1)
feature_view = fs.get_or_create_feature_view(
    name="target_feature_view",
    version=1,
    description='visualizar a data dos clients',
    query=cheque_fg.select(["output"]),
)

In [6]:
import pandas as pd
import hsfs
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

# Function to build expectation suite for feature groups
def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): Name of the expectation suite.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    expectation_suite = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    
    # Add expectations based on the feature group
    if feature_group == 'client_features':
        # Add expectations for client features
        pass  # Add your expectations here
    elif feature_group == 'health_features':
        # Add expectations for health features
        pass  # Add your expectations here
    elif feature_group == 'target_features':
        # Add expectations for target features
        pass  # Add your expectations here
    
    return expectation_suite


# Function to perform data validation against expectation suite
def validate_data(df, suite_name, feature_group):
    """
    Validate data against the expectation suite.
    
    Args:
        df (pd.DataFrame): DataFrame to validate.
        suite_name (str): Name of the expectation suite.
        feature_group (str): Feature group used to construct the expectations.
        
    Returns:
        Validation result.
    """
    print(f"Validating data against {suite_name} for {feature_group}...")
    expectation_suite = build_expectation_suite(suite_name, feature_group)
    # Perform validation using Great Expectations
    # Your validation logic here...
    validation_result = True  # Placeholder for validation result
    print("Validation complete.")
    return validation_result


# Function to load data into Feature Store
def to_feature_store(data: pd.DataFrame, group_name: str, feature_group_version: int, description: str,
                     group_description: dict, validation_expectation_suite: ExpectationSuite, SETTINGS: dict):
    """
    This function takes in a pandas DataFrame and a validation expectation suite,
    performs validation on the data using the suite, and then saves the data to a
    feature store in the feature store.

    Args:
        data (pd.DataFrame): DataFrame with the data to be stored
        group_name (str): Name of the feature group.
        feature_group_version (int): Version of the feature group.
        description (str): Description for the feature group.
        group_description (dict): Description of each feature of the feature group.
        validation_expectation_suite (ExpectationSuite): group of expectations to check data.
        SETTINGS (dict): Dictionary with the settings definitions to connect to the project.
        
    Returns:
        A dictionary with the feature view version, feature view name and training dataset feature version.
    """
    print(f"Loading data into Feature Store for {group_name}...")
    # Connect to feature store
    connection = hsfs.connection(
        host="c.app.hopsworks.ai/p/766100/view",
        project="MLOps_Hospital",
        engine="python",
        api_key_value="1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24")
    fs = connection.get_feature_store()
    
    # Get or create feature group
    feature_group = fs.get_feature_group(group_name, version=feature_group_version)
    
    # Perform data validation
    validation_result = validate_data(data, validation_expectation_suite.expectation_suite_name, group_name)
    
    if validation_result:
        print("Data validation successful. Inserting data into Feature Store...")
        # Insert data into feature store
        feature_group.insert(
            features=data,
            overwrite=False,
            write_options={"wait_for_job": True},
        )
        
        # Add feature descriptions
        for description in group_description:
            feature_group.update_feature_description(description["name"], description["description"])
        
        # Update statistics
        feature_group.statistics_config = {
            "enabled": True,
            "histograms": True,
            "correlations": True,
        }
        feature_group.update_statistics_config()
        feature_group.compute_statistics()
        print("Data successfully loaded into Feature Store.")
    else:
        print("Data validation failed. Aborting data loading process.")
    
    return validation_result  # Return validation result or any other necessary information

In [8]:
import hsfs

# Connect to the Feature Store
connection = hsfs.connection(
    host="c.app.hopsworks.ai/p/766100/view",
    project="MLOps_Hospital",
    engine="python",
    api_key_value="1DPveB1Vmqt3lwpB.grlIqLltPHHcGN9vj5jM3AlfP4llFrEs32SUcZdmJbjhdZdfhY5kJEO1fu9lOU24"
)
fs = connection.get_feature_store()

# List of feature groups to explore
feature_groups_to_explore = ["client_feature_group", "health_feature_group", "target_feature_group"]

# 1. Explore Feature Groups
for group_name in feature_groups_to_explore:
    try:
        feature_groups = fs.get_feature_groups(name=group_name)

        for feature_group in feature_groups:
            print("Feature Group Name:", feature_group.name)
            print("Feature Group Description:", feature_group.description)
            print("Feature Group Version:", feature_group.version)
            print("Features in Feature Group:", [f.name for f in feature_group.features])
            print("--------------------------------------------")
    except Exception as e:
        print(f"An error occurred while accessing feature group {group_name}: {e}")

# 2. Query Features
for group_name in feature_groups_to_explore:
    try:
        feature_group = fs.get_feature_group(group_name)  # Assuming the latest version
        features = feature_group.read()

        # Print the first few rows of the features
        print(f"First few rows of features from {group_name}:")
        print(features.head())
    except Exception as e:
        print(f"An error occurred while querying features from feature group {group_name}: {e}")

Connected. Call `.close()` to terminate connection gracefully.
Feature Group Name: client_feature_group
Feature Group Description: 
Feature Group Version: 1
Features in Feature Group: ['index', 'sex', 'age']
--------------------------------------------
Feature Group Name: health_feature_group
Feature Group Description: 
Feature Group Version: 1
Features in Feature Group: ['cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall', 'index']
--------------------------------------------
Feature Group Name: target_feature_group
Feature Group Description: 
Feature Group Version: 1
Features in Feature Group: ['index', 'output']
--------------------------------------------
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 
First few rows of features from client_feature_group:
   index  sex  age
0    104    1   50
1    159    1   56
2    186    1   60
3    203    1   68
4     26    1   59
Finished: Reading data from Hopswor

### Não é normal termos todas as variaveis significativas. Pode ser por causa do nosso dataset ser do kaggle.

### Low KS Value + High P-Value: Good, no significant data drift.
### High KS Value + Low P-Value: Concerning, indicates significant data drift.

### Features with significant data drift: Investigate the cause of the drift. This might involve looking at changes in data collection methods, new patterns in the data, or external factors influencing the data.

### Mitigate Drift: Consider retraining your model with more recent data or applying techniques to handle the drift, such as data preprocessing adjustments.